In [1]:
import os
ROOT_DIR = "/home/ebr/projects/inundation-emulator"
os.chdir(ROOT_DIR)

In [ ]:
from src.emulator import Emulator, DataReader
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# Load pretrained emulator.

GENERATED_DIR = "/home/ebr/projects/inundation-emulator/generated"
TOPO_FILE = '/home/ebr/data/PTHA2020_runs_UMA/Catania/C_CT.grd'
TOPO_MASK = '/home/ebr/data/PTHA2020_runs_UMA/Catania/ct_mask.txt'
TRAIN_SCENARIOS ="/home/ebr/data/PTHA2020_runs_UMA/train_591/scenarios.txt"
TRAIN_DIR = '/home/ebr/data/PTHA2020_runs_UMA/train_591'
VALIDATION_SCENARIOS = '/home/ebr/data/PTHA2020_runs_UMA/test/scenarios.txt'
VALIDATION_DIR = "/home/ebr/data/PTHA2020_runs_UMA/test"

RUNDIR = "/home/ebr/projects/inundation-emulator/generated/emulator_20250123_220604"
EPOCH_CHECKPOINT = 500

emulator = Emulator(GENERATED_DIR, TOPO_FILE, TOPO_MASK, rundir=RUNDIR, epoch_checkpoint=EPOCH_CHECKPOINT)

In [4]:
import numpy as np
topomask_file = "/home/ebr/projects/inundation-emulator/article_data/ct_mask.txt"


with open(topomask_file, 'r') as file:
    lines = file.readlines()

# Convert each line to a boolean (True for "true", False for "false")
topomask = np.array([
    [element.strip().lower() == 'true' for element in line.split()]
    for line in lines
], dtype=bool).T

In [ ]:
topomask.shape

In [ ]:
from datetime import datetime

scenarios_file = TRAIN_SCENARIOS
input_dir = TRAIN_DIR
predictions_dir = "/home/ebr/projects/inundation-emulator/predictions"
batch_size = 10

# Create output folder
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = os.path.join(predictions_dir, emulator.id, f"preds_{timestamp}")
os.makedirs(output_dir, exist_ok=True)

with open(scenarios_file, 'r') as file:
    nr_of_scenarios = sum(1 for line in file if line.strip())
    print(f"Number of scenarios for prediction: {nr_of_scenarios}")

reader = DataReader( 
        scenarios_file=scenarios_file,
        pois=emulator.pois,
        datadir=input_dir,
        topofile=emulator.topofile,
        topomask_file=emulator.topomask_file,
        shuffle_on_load=False,
        target=True,
        reload=False
)

output_signature = (
    tf.TensorSpec(shape=(emulator.n_pois, emulator.input_time_steps), dtype=tf.float32),           # eta
    tf.TensorSpec(shape=(reader.topomask.sum()), dtype=tf.float32),         # No flow_depth
    tf.TensorSpec(shape=(), dtype=tf.string)                               # scenario
)

dataset = tf.data.Dataset.from_generator(
        generator=reader.generator,
        output_signature=output_signature
).batch(batch_size)


In [ ]:
preds = np.zeros((batch_size, *reader.topomask.shape))
flow_depths = np.zeros((batch_size, *reader.topomask.shape))

for eta, flow_depth, scenario_id in dataset:
    # Make predictions
    #preds[:, topomask] = emulator.model(eta, training=False)
    preds[:, topomask] = emulator.model.predict(eta)
    flow_depths[:,topomask] = flow_depth
    break


In [ ]:
emulator.model.predict(eta).shape

In [ ]:
plt.imshow(np.flip(flow_depths[1,:], axis=0))
plt.colorbar()

In [ ]:
plt.imshow(np.flip(preds[1,:], axis=0))
plt.colorbar()

In [ ]:
from netCDF4 import Dataset
from shutil import copyfile

for i in range(10):
    # Create a NetCDF file for each scenario
    scenario_id_str = scenario_id.numpy().decode('utf-8')  # Decode scenario_id to string
    pred_file = os.path.join(output_dir, f"{scenario_id_str}_CT_10m_PR.nc")
    
    with Dataset(pred_file, mode='w', format="NETCDF4") as ds:from netCDF4 import Dataset

# Input and Output file paths
input_file = "input.nc"
output_file = "output.nc"

# Open the input file in read mode
with Dataset(input_file, "r") as src:

    # Create the output file
    with Dataset(output_file, "w", format="NETCDF4") as dst:

        # Copy dimensions from the input file
        for dim_name, dim in src.dimensions.items():
            dst.createDimension(dim_name, (len(dim) if not dim.isunlimited() else None))

        # Copy variables related to spatial grid
        grid_vars = ["grid_lat", "grid_lon", "lat", "lon"]  # Adjust this based on your input file
        for var_name in grid_vars:
            if var_name in src.variables:
                src_var = src.variables[var_name]
                
                # Create variable in output file with same properties
                dst_var = dst.createVariable(var_name, src_var.datatype, src_var.dimensions)
                
                # Copy attributes
                dst_var.setncatts({attr: src_var.getncattr(attr) for attr in src_var.ncattrs()})
                
                # Copy data
                dst_var[:] = src_var[:]

        print("Successfully copied dimensions and spatial grid variables.")

        
        

In [ ]:

    # Create an xarray DataArray for the prediction
    data = xr.DataArray(
        preds[i,:],
        dims=topomask.shape,
        name="prediction",
    )

    # Add metadata or attributes if necessary
    data.attrs["scenario"] = scenario_id_str
    data.attrs["description"] = "Model predictions for scenario."
    data.attrs[]

    # Save to NetCDF
    data.to_netcdf(file_path)
    print(f"Saved predictions for scenario '{scenario_id_str}' to {file_path}")